# Lab 3: Solve the Maze using A* - Group 11 - Sebastian Danielsson (h21sebda)

## Generate maze function

In [ ]:
from pylab import *
import numpy

def make_maze(w, h):
    vis = [[0] * w + [1] for _ in range(h)] + [[1] * (w + 1)]
    ver = [["10"] * w + ['1'] for _ in range(h)] + [[]]
    hor = [["11"] * w + ['1'] for _ in range(h + 1)]
 
    def walk(x, y):
        vis[y][x] = 1
 
        d = [(x - 1, y), (x, y + 1), (x + 1, y), (x, y - 1)]
        shuffle(d)
        for (xx, yy) in d:
            if vis[yy][xx]: continue
            if xx == x: hor[max(y, yy)][x] = "10"
            if yy == y: ver[y][max(x, xx)] = "00"
            walk(xx, yy)
 
    walk(numpy.random.randint(w), numpy.random.randint(h))
 
    s = ""
    for (a, b) in zip(hor, ver):
        s += ''.join(a + ['\n'] + b + ['\n'])
        
    M=[]
    for line in s.split("\n"):
        if line!="":
            R=[]
            for e in line:
                R.append(int(e))
            M.append(R)
    return M

## A* functions

In [ ]:
# Get neighbors of a square
def get_neighbors(maze, square):
    x, y = square
    neighbors = []
    if x > 0 and maze[x - 1][y] == 0:
        neighbors.append((x - 1, y))
    if x < len(maze) - 1 and maze[x + 1][y] == 0:
        neighbors.append((x + 1, y))
    if y > 0 and maze[x][y - 1] == 0:
        neighbors.append((x, y - 1))
    if y < len(maze[0]) - 1 and maze[x][y + 1] == 0:
        neighbors.append((x, y + 1))
    return neighbors

# A* algorithm
def a_star(maze, start, end):
    # List of squares to visit
    queue = [start]
    # Keep track of the parents of each square
    parents = {start: None}
    # Dictionary with the cost from start to each square
    costs = {start: 0}
    # List or set to store visited squares
    visited = []

    # Repeat until we have visited all squares
    while queue:
        # Sort the queue by cost + heuristic
        queue.sort(key=lambda x: costs[x] + abs(end[0] - x[0]) + abs(end[1] - x[1]))

        # Get the square with the lowest cost + heuristic
        current = queue.pop(0)

        # Add the current square to the visited list or set
        visited.append(current)

        # Check if we have reached the end
        if current == end:
            # Save the path
            path = []
            while current:
                path.append(current)
                current = parents[current]
            return path[::-1], visited, costs

        neighbors = get_neighbors(maze, current)

        # Get cost of reaching each neighbor
        for neighbor in neighbors:
            cost = costs[current] + 1

            # Update the cost and parent of the neighbor if the cost is missing or lower than the current known cost
            if neighbor not in costs or cost < costs[neighbor]:
                costs[neighbor] = cost
                parents[neighbor] = current
                queue.append(neighbor)

    # Return none if no path to the end is found
    return None

## Create a maze and plot the mission

In [ ]:
# Configure properties for the maze
maze_width=8
maze_height=8
startLocation=(1,1)
targetLocation=(maze_height*2-1,maze_width*2-1)
# Create the maze
maze=numpy.array(make_maze(maze_width,maze_height))

# Create a function to plot the maze
def plot_maze(maze, start, end, maze_width, maze_height):
    # Plot the maze
    plt.imshow(maze, cmap='binary')
    # Show tick for each square
    plt.xticks(range(0, maze_width*2))
    plt.yticks(range(0, maze_height*2))
    # Show the start and target locations
    plt.plot(start[1], start[0], 'gs')
    plt.plot(end[1], end[0], 'rs')

# Check if the start and target locations are valid
if maze[startLocation[0]][startLocation[1]]==1:
    print('Start location is not valid =', maze[1][1])
else:
    print('Start location is valid =', maze[1][1])
if maze[targetLocation[0]][targetLocation[1]]==1:
    print('Target location is not valid =', maze[15][15])
else:
    print('Target location is valid =', maze[15][15])

# Plot the mission
plt.figure(figsize=[maze_width/2,maze_height/2])
plot_maze(maze, startLocation, targetLocation, maze_width, maze_height)
plt.show()

In [ ]:
# Find the best path with A*
generated_path, visited, costs = a_star(maze, startLocation, targetLocation)

# Plot the visited squares and the best path
plt.figure(figsize=[maze_width*2,maze_height*2])

plt.subplot(1, 3, 1)
plot_maze(maze, startLocation, targetLocation, maze_width, maze_height)
plot(transpose(generated_path[1:-1])[1],transpose(generated_path[1:-1])[0],'k.')

plt.subplot(1, 3, 2)
plot_maze(maze, startLocation, targetLocation, maze_width, maze_height)
plot(transpose(visited[1:-1])[1],transpose(visited[1:-1])[0],'k.')

plt.subplot(1, 3, 3)

# Set illegal moves and unexplored squares to -1
cost_array = [[-1 for _ in range(len(maze[0]))] for _ in range(len(maze))]

# Add the costs of the visited squares
for square, cost in costs.items():
    x, y = square
    cost_array[x][y] = cost

# Plot the maze with the costs
imshow(cost_array,cmap='Spectral')
# Show tick for each square
plt.xticks(range(0, maze_width*2))
plt.yticks(range(0, maze_height*2))
# Show the start and target locations
plt.plot(startLocation[1], startLocation[0], 'gs')
plt.plot(targetLocation[1], targetLocation[0], 'rs')

# Show the figure
plt.show()